In [1]:
import selenium
from selenium import webdriver
import os
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
print('-*-'*50)
print('2021-2 Marketing analytics')
print('디시인사이드 갤럭시 커뮤니티 게시글+댓글 크롤러')
print('-*-'*50)

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'
os.makedirs(f_dir+s+'-'+'디시_갤럭시_커뮤니티_게시글과_댓글')
os.chdir(f_dir+s+'-'+'디시_갤럭시_커뮤니티_게시글과_댓글')
cnt = int(input('몇 건 게시물을 수집하시겠습니까?'))
print(f'수집 목표 : 데이터 {cnt}개')

driver_path = '/Users/kibeomkim/Desktop/chromedriver_intel/m1_chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
driver = webdriver.Chrome(driver_path, options=options)
driver.get('https://gall.dcinside.com/mgallery/board/lists?id=galaxy')
s_time = time.time()
time.sleep(random.randint(4,8))

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
2021-2 Marketing analytics
디시인사이드 갤럭시 커뮤니티 게시글+댓글 크롤러
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
수집 목표 : 데이터 3개


In [3]:
# part 1: 페이지 파싱해서 url 주소만 전부 수집. 리스트에 추가하기 
page_no = 1
url_list = []
no = 0

while True: # 페이지 전환
    loaded_page = driver.page_source
    soup = BeautifulSoup(loaded_page, 'html.parser')
    tr_list = soup.find_all('tr', 'ub-content us-post')
    
    for tr in tr_list : 
        writing_type = tr.find('td', 'gall_subject').get_text()
        if writing_type == '공지' :  # 공지글인지 아닌지 검사하기 
            continue # 공지글이면 패스 
        else : 
            url = tr.find('a')['href']
            no += 1
            if no > cnt : 
                break
            url_list.append(url)
    if no > cnt : 
        break
    page_no += 1 # 다음 페이지 번호 

    for scroll in range(4) : # 스크롤 다운
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        time.sleep(4)

    try : 
        time.sleep(3)
        driver.find_element_by_link_text(str(page_no)).click()
        time.sleep(random.randint(5,8))
    except : 
        try : 
            time.sleep(3)
            driver.find_element_by_link_text('다음').click() # 다음 페이지로 넘어가기 
            time.sleep(5)

            driver.find_element_by_link_text(str(page_no)).click() # 원래 넘어가야 하는 페이지 번호 다시 클릭하기 
            time.sleep(random.randint(5,9))
        except : 
            print('다음 페이지가 없습니다')
            break
driver.quit()
time.sleep(2)

In [4]:
# part 2 : 페이지 순차 접속 후 게시 정보, 본문 내용, 그리고 댓글 정보 수집하기 
index_list = []
index_no = 1

title_list = []
writer_list = []
posting_date_list = []
contents_list = []

cmt_index_list = []
cmt_nick_name_list = []
cmt_contents_list = []
cmt_time_list = []

cmt_no_list = []
reply_index_list = []
reply_nick_name_list = []
reply_contents_list = []
reply_date_list = []


for url in url_list : 
    # 크롬 드라이버 호출
    options = webdriver.ChromeOptions()
    options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
    driver = webdriver.Chrome(driver_path, options=options)

    # 페이지 접속 
    driver.get('https://gall.dcinside.com'+url)
    time.sleep(random.randint(5,9)) # 페이지 로드 대기 

    # 페이지 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 제대로 된 페이지인지 검사 
    try : 
        soup.find('span', 'title_subject').get_text()
        pass
    except : # 찾을 수 없는 페이지이면 건너뛰기 
        driver.quit()
        continue

    # 게시물 인덱스 넘버 추가 
    index_list.append(index_no)

    # 게시글 제목 수집 
    title = soup.find('span', 'title_subject').get_text().replace(' ', "")
    title_list.append(title)
    print('-'*100)
    print(f'게시글 제목 : {title}')

    # 게시자 수집 
    writer = soup.find('span', 'nickname in').get_text().replace(' ', "")
    writer_list.append(writer)
    print(f'게시자 : {writer}')

    # 게시 일자 수집 
    posting_date = soup.find('span', 'gall_date').get_text().replace(' ', "")
    posting_date_list.append(posting_date)
    print(f'게시 일자 : {posting_date}')

    # 본문 내용 수집 
    contents = soup.find('div', 'write_div').get_text().replace('\n', "")
    contents_list.append(contents)
    print(f'본문 내용 : {contents}')

    for scroll in range(2) : # 스크롤 다운
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        time.sleep(3)

    # 댓글 & 대댓글 수집 
    try : # 댓글이 있으면 
        driver.find_element_by_class_name('cmt_list')
        
        cmt_list = soup.find('ul', 'cmt_list').find_all('li', 'ub-content')
        print('*'*100)

        for li in cmt_list : 

            cmt_index_list.append(index_no) # 어느 게시글 댓글인지 표시 

            # 작성자 닉네임 수집
            try : 
                cmt_nick_name = li.find('div', 'cmt_nickbox').get_text().replace('\n', '').replace(' ', '')
                if len(cmt_nick_name) == 0 : # 작성자 이름 태그 있는데, 작성자 이름도 있는지 없는지 검사 
                    cmt_nick_name_list.append(' ')
                    print(f'댓글 작성자 닉네임 : 정보 없음')
                else : 
                    cmt_nick_name_list.append(cmt_nick_name)
                    print(f'댓글 작성자 닉네임 : {cmt_nick_name}')
            except : 
                cmt_nick_name_list.append(' ')
                print(f'댓글 작성자 태그 없음')
            
            # 댓글 내용 수집 
            # 댓글 내용 태그 있는지 검사 
            try : # 일반 댓글인가 
                cmt_contents = li.find('div','clear cmt_txtbox btn_reply_write_all').get_text().replace('\n', '').replace(' ', '')
                if len(cmt_contents) == 0 : 
                    cmt_contents_list.append(' ')
                    print('댓글내용 : 정보 없음')
                else : 
                    cmt_contents_list.append(cmt_contents)
                    print(f'댓글내용 : {cmt_contents}')
            except : 
                try : # 대댓글인가
                    cmt_contents = li.find('div', 'clear cmt_txtbox').get_text().replace('\n', "").replace(' ', '')
                    if len(cmt_contents) == 0 : 
                        cmt_contents_list.append(' ')
                        print('댓글내용 : 정보 없음')
                    else : 
                        cmt_contents_list.append(cmt_contents)
                        print(f'댓글내용 : {cmt_contents}')
                except : 
                    cmt_contents_list.append(' ')
                    print('댓글 내용 태그 없음')
                

            # 댓글 게시 시간 수집 
            try : 
                cmt_time = li.find('span', 'date_time').get_text().replace('\n', '')
                if len(cmt_time) == 0 : 
                    cmt_time_list.append(' ')
                    print(f'댓글내용 : 정보 없음')
                else : 
                    cmt_time_list.append(cmt_time)
                    print(f'댓글 게시 시간 : {cmt_time}')
            except : 
                cmt_time_list.append(' ')
                print(f'댓글 게시 시간 태그 없음')

        print('*'*100) # 댓글 닫기 
        print('-'*100) # 본문 닫기 
        index_no += 1
        driver.quit() 
        
    except : # 댓글이 없으면 
        print('댓글 없음')
        print('-'*100) # 본문 닫기 

        index_no += 1
        driver.quit() # 댓글 없다. 다음 페이지로 가자. 


----------------------------------------------------------------------------------------------------
게시글 제목 : 형들그럼마지막으로하나만물어볼게ㅠ
게시자 : ㅇㅇ
게시 일자 : 2021.11.0901:42:04
본문 내용 : 주사율은 크게 신경 안 쓰는데 패널 차이가 좀 커서 고민이네..갤탭 7 fe lcd 갤탭 7+ oled 저거 하나 차이로 30만원 넘게 지출할 가치가 있을까?어떤 분은 별 차이 안 나니까 신경 쓰지 말라고 그래서.. 
댓글 없음
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
게시글 제목 : 솔직히출시당시디자인역대급이였던폰.jpg
게시자 : ㅇㅇ
게시 일자 : 2021.11.0901:25:27
본문 내용 : 근데 폭탄임 ㅠㅠ 
댓글 없음
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
게시글 제목 : 게임을안하니성능에연연을안하게됐다
게시자 : ㅇㅇ
게시 일자 : 2021.11.0901:22:58
본문 내용 : 이게 다 돈독오른 새끼들의 개좃같은 과금유도 덕임 
****************************************************************************************************
댓글 

In [5]:
# part 3: 리스트에 수집한 내용 csv 파일로 저장하기 

# 본문 데이터 
contents_table = pd.DataFrame()
contents_table['게시글 제목'] = title_list
contents_table['게시자 이름'] = writer_list
contents_table['게시글 게시 일자'] = posting_date_list
contents_table['게시글 본문'] = contents_list
contents_table['게시글 식별번호'] = index_list
fc_name = '디시인사이드_갤럭시갤러리_게시글_본문데이터.csv'
fx_name = '디시인사이드_갤럭시갤러리_게시글_본문데이터.xlsx'
contents_table.to_csv(fc_name, encoding='utf-8-sig', index=False)
contents_table.to_excel(fx_name, index=False)

# 댓글 데이터 
cmt_table = pd.DataFrame()
cmt_table['게시글 식별번호'] = cmt_index_list 
cmt_table['댓글 작성자 닉네임'] = cmt_nick_name_list 
cmt_table['댓글 본문 내용'] = cmt_contents_list 
cmt_table['댓글 작성 일자'] = cmt_time_list 
fc_name = '게시글_댓글데이터.csv'
fx_name = '게시글_댓글데이터.xlsx'
cmt_table.to_csv(fc_name, encoding='utf-8-sig', index=False)
cmt_table.to_excel(fx_name, index=False)

e_time = time.time()
total_time = e_time - s_time 
print('='*100)
print(f'총 {int(index_no)-1} 건 만큼 게시글을 수집했습니다')
print(f'총 소요시간은 {total_time} 초 입니다')
print('='*100)

총 3 건 만큼 게시글을 수집했습니다
총 소요시간은 151.76919603347778 초 입니다
